In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
from keras.models import load_model
K.set_image_data_format('channels_first')

import pickle
import cv2
import os.path
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utility import *
from webcam_utility import *

Using TensorFlow backend.


In [2]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # triplet formula components
    pos_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[1])) )
    neg_dist = tf.reduce_sum( tf.square(tf.subtract(y_pred[0], y_pred[2])) )
    basic_loss = pos_dist - neg_dist + alpha
    
    loss = tf.maximum(basic_loss, 0.0)
   
    return loss

In [3]:
FRmodel = load_model('models/model.h5', custom_objects={'triplet_loss': triplet_loss})

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def ini_user_database():
    if os.path.exists('database/user_dict.pickle'):
        with open('database/user_dict.pickle', 'rb') as handle:
            user_db = pickle.load(handle)   
    else:
        user_db = {}
     
    return user_db

In [5]:
def add_user_img_path(user_db, FRmodel, name, img_path):
    if name not in user_db: 
        user_db[name] = img_to_encoding(img_path, FRmodel)
        # save the database
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print('User ' + name + ' added successfully')
    else:
        print('The name is already registered! Try a different name.........')

In [6]:
def add_user_webcam(user_db, FRmodel, name):
    face_found = detect_face(user_db, FRmodel)

    if face_found:
        resize_img("saved_image/1.jpg")
        if name not in user_db: 
            add_user_img_path(user_db, FRmodel, name, "saved_image/1.jpg")
        else:
            print('The name is already registered! Try a different name.........')
    else:
        print('There was no face found in the visible frame. Try again...........')

In [7]:
def delete_user(user_db, name):
    popped = user_db.pop(name, None)
    
    if popped is not None:
        print('User ' + name + ' deleted successfully')
        # save the database
        with open('database/user_dict.pickle', 'wb') as handle:
                pickle.dump(user_db, handle, protocol=pickle.HIGHEST_PROTOCOL)
    elif popped == None:
        print('No such user !!')

In [8]:
user_db = ini_user_database()

In [101]:
add_user_img_path(user_db, FRmodel, "Akshat", "images/3.jpg")

The name is already registered! Try a different name.........


In [10]:
add_user_webcam(user_db, FRmodel, "Prakhar") 

The name is already registered! Try a different name.........


In [42]:
delete_user(user_db, "Akshat")

User Akshat deleted successfully


In [16]:
def find_face(image_path, database, model, threshold = 0.6):
    encoding = img_to_encoding(image_path, model)
    
    min_dist = 99999
    for name in database:
        dist = np.linalg.norm(np.subtract(database[name], encoding) )
        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > threshold:
        print("User not in the database.")
        identity = 'Unknown Person'
    else:
        print ("Hi! " + str(identity) + ", L2 distance: " + str(min_dist))
        
    return min_dist, identity

In [17]:
def do_face_recognition(user_db, FRmodel, threshold = 0.7, save_loc = "saved_image/1.jpg"):
    face_found = detect_face(user_db, FRmodel)
    
    if face_found:
        resize_img("saved_image/1.jpg")
        min_dist, identity  = find_face("saved_image/1.jpg", user_db, FRmodel, threshold)
    else:
        print('There was no face found in the visible frame. Try again...........')
    return identity    

In [18]:
identity  = do_face_recognition(user_db, FRmodel, 0.6, "saved_image/1.jpg")

Hi! Prakhar, L2 distance: 0.4585928


Prakhar


In [43]:
detect_face_realtime(user_db, FRmodel, threshold = 0.7)

**************** Enter "q" to quit **********************
Welcome Prakhar!
distance:0.64269394
Welcome prakhar!
distance:0.5596692
Welcome Prakhar!
distance:0.6067535
Welcome prakhar!
distance:0.5537436
Welcome Prakhar!
distance:0.5565138
Welcome akshat!
distance:0.49492627
Unknown user detected !
distance:0.73857933
Welcome prakhar!
distance:0.6354592
Welcome prakhar!
distance:0.6579643
Welcome prakhar!
distance:0.6066288
Welcome Prakhar!
distance:0.46012935
Welcome Prakhar!
distance:0.5061029
Welcome Prakhar!
distance:0.45667928
Welcome Prakhar!
distance:0.49003217
Unknown user detected !
distance:0.70944554
Welcome prakhar!
distance:0.5770175
Welcome prakhar!
distance:0.65907735
Welcome prakhar!
distance:0.65816915
Welcome prakhar!
distance:0.67391497
Welcome prakhar!
distance:0.6417995
Unknown user detected !
distance:0.708098
Unknown user detected !
distance:0.7044114
Unknown user detected !
distance:0.70236963
Welcome Saharsh!
distance:0.6556032
Welcome Saharsh!
distance:0.658816


q
